# Sessió 4: PyTorch & Búsqueda aproximada

## **NOM**: ####

## **NIU**: ####

En aquesta sessió continuarem utilitzant pytorch per definir i evaluar diferents models amb xarxes neuronals.

* Durant la classe, repasarem el codi aqui mostrat i veurem què podem provar.
* **A casa**
 * Apartat A. Búsqueda Brute Force **(2pts)**
 * Apartat B. Búsqueda Aproximada **(3pts)**
 * Apartat C. Net Encoding. Búsqueda Brute Force **(2pts)**
 * Apartat D. Net Encoding. Búsqueda Aproximada **(3pts)** 
 

Treballarem sobre la base de dades [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist). És similar a la base de dades de MNIST, un dataset clàssic en la visió per computador. Són imatges de 28x28 pixels i en escala de grisos. El original disposa de 60.000 imatges de entrenament i 10.000 de test. Està anotat en 10 categories diferents:

|Id|Nom|
|:-:|:--|
|0|T-shirt/top
|1|Trouser |
|2|Pullover|
|3|Dress|
|4|Coat|
|5|Sandal|
|6|Shirt|
|7|Sneaker|
|8|Bag |
|9|Ankle boot|

<img src="https://github.com/zalandoresearch/fashion-mnist/raw/master/doc/img/fashion-mnist-sprite.png" width="60%">


### Imports

In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
import tqdm
import sys
import time
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

### Parameters

Aqui definim els principals parametres que poden fer variar el metode

In [2]:
quick_experiment = True  # reduce the number of training and testing samples for a quick check
epochs_in_quick = 2      # number of epochs in the fast experiment
n_train_in_quick = 60000  # number of training images in the fast experiment
n_test_in_quick = 1000    # number of testing images in the fast experiment

epochs = 5             # number of epochs to train (default: 14)
learning_rate = 0.001   # learning rate (default: 0.001-[0.01]-0.1)

Si fem un experiment rapid, quin % del total estem agafant:

In [3]:
total_training_data = 60000 # no tocar. només serveix per visualitzar
total_testing_data = 10000  # no tocar. només serveix per visualitzar

if quick_experiment:
    total_training_data = 100.*n_train_in_quick/total_training_data
    total_testing_data = 100.*n_test_in_quick/total_testing_data
else:
    total_training_data = 100
    total_testing_data = 100
    
print("TRAINING USED:  {:.1f}%\nTESTING  USED:  {:.1f}%".format(total_training_data, total_testing_data))

TRAINING USED:  100.0%
TESTING  USED:  10.0%


En principi els següents parametres son força estandard i no caldria tocar-los gaire si no sabeu el que volen dir.

In [4]:
batch_size = 100        # number of samples during training
test_batch_size = 1000  # number of samples for test 

no_cuda = True          # disables CUDA training
dry_run = False         # quickly check a single pass
seed = 1                # random seed (default: 1)
log_interval = 50       # how many batches to wait before logging training status
save_model = False      # For Saving the current Model


# Check if cuda is available
use_cuda = not no_cuda and torch.cuda.is_available()
print(f"USING CUDA: {use_cuda}")
torch.manual_seed(seed)

# define the device where to compute (cpu or gpu)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)


USING CUDA: False


### Models

In [5]:
class FashionCNN(nn.Module):

    def __init__(self):
        super(FashionCNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(in_features=64 * 6 * 6, out_features=600)
        self.flatten = nn.Flatten()
        self.drop1 = nn.Dropout(0.25)
        self.drop2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.flatten(out)
        out = self.drop1(out)
        out = self.fc1(out)
        out = self.drop2(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.softmax(out)

        return out
    
class FashionCNN_feature_extraction(FashionCNN):
    def __init__(self):
        super(FashionCNN_feature_extraction, self).__init__()

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.flatten(out)
        out = self.drop1(out)
        out = self.fc1(out)
        out = self.drop2(out)
        out = self.fc2(out)
#        out = self.fc3(out)
#        out = self.softmax(out)

        return out

### Funcions auxiliars

In [6]:
def visualize_confusion_matrix(y_pred, y_real):
    # mostra la matriu de confusió
    cm = confusion_matrix(y_real, y_pred)
    plt.subplots(figsize=(10, 6))
    sns.heatmap(cm, annot = True, fmt = 'g')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()

def calcular_parametres_del_model(current_model):
    pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("# trainable parameters: {:,}".format(pytorch_total_params))
    return pytorch_total_params

def calculate_parameters_and_flops(current_model):
    from thop import profile
    test_input = torch.randn(1, 1, 28, 28)
    macs, params = profile(current_model, inputs=(test_input,))  # multiply accumulate operation (GFLOPS = 2 * GMACS)
    # normalment, en gpus i exemples reals, es parlaria minim de Gigaflops.
    # print("%s | %.2f Params(M) | %.3f FLOPs(G)" % (current_model._get_name(), params / (1000 ** 2), macs / (1000 ** 3)))
    print("%s | %.2f Params(M) | %.3f FLOPs(M)" % (current_model._get_name(), params / (1000 ** 2), macs / (1000 ** 2)))
    return macs, params
    
def mostra_estructura_model_torchviz(current_model):
    from torchviz import make_dot
    test_input = torch.randn(1, 1, 28, 28)
    return make_dot(current_model(test_input), params=dict(current_model.named_parameters()))
    
    
def mostra_estructura_model_hiddenlayer(current_model):
    import hiddenlayer as hl
    test_input = torch.randn(1, 1, 28, 28)
    hl_graph = hl.build_graph(current_model, test_input)
    return hl_graph


### Train loop

In [7]:
def train(model, device, train_loader, optimizer, criterion):
    losses = []
    model.train()
    t = tqdm.tqdm(enumerate(train_loader), total=len(train_loader))
    t.set_description('Train')
    for batch_idx, (data, target) in t:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        t.set_postfix(loss=loss.item())

    return losses

### Test loop

In [8]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    totals = 0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        t = tqdm.tqdm(test_loader, total=len(test_loader))
        t.set_description('Test ')
        for data, target in t:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() * data.shape[0]  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            totals += len(target)
            t.set_postfix(loss=test_loss/totals, accuracy=100.*correct/totals)
            all_preds.extend(np.asarray(pred))
            all_targets.extend(np.asarray(target))

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    visualize_confusion_matrix(all_preds, all_targets)    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    return test_loss, accuracy

### Experiment Loop

In [9]:
def experiment(model, device, loss, optimizer, train_loader, test_loader, name='', save_model=False):
    init_time = time.time()
    losses_train = []
    losses_test = []
    accuracies_test = []
    print('--'*50)
    print('STARTING EXPERIMENT {}'.format(name))
    print('--'*50)
    
    model.to(device)
    print("CHECKING INITIAL TEST LOSS (with random weights..)")
    loss_test_epoch, accuracy_epoch = test(model, device, test_loader, loss)
    losses_test.append(loss_test_epoch)
    accuracies_test.append(accuracy_epoch)
    
    for epoch in range(1, epochs + 1):
        print ("EPOCH {}".format(epoch))
        sys.stdout.flush()
        loss_train_epoch = train(model, device, train_loader, optimizer, loss)
        loss_test_epoch, accuracy_epoch = test(model, device, test_loader, loss)

        losses_train.extend(loss_train_epoch)
        losses_test.append(loss_test_epoch)
        accuracies_test.append(accuracy_epoch)

    plt.plot(range(len(losses_train)), 
             losses_train, label="Training Loss")
    plt.plot(range(0, len(losses_train)+1, int(len(losses_train)/(len(losses_test)-1))), 
             losses_test, label="Test Loss")
    plt.legend()
    plt.show()
    
    elapsed = time.time()-init_time

    if save_model:
        torch.save(model.state_dict(), "fashion_mnist_cnn.pt")

    print ("ELAPSED TIME: {:.1f}s".format(elapsed))

    return losses_train, losses_test, accuracies_test, elapsed

### Preparar les dades d'entrenament
Aquí, mantindrem el codi de la sessió anterior

In [10]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,))
])
dataset_train = torchvision.datasets.FashionMNIST('../data', train=True, download=True, transform=transform)
dataset_val = torchvision.datasets.FashionMNIST('../data', train=False, transform=transform)

if quick_experiment:
    epochs = epochs_in_quick
    n_not_used_for_train_in_quick = len(dataset_train) - n_train_in_quick
    dataset_train, dataset_train_not_used = torch.utils.data.random_split(dataset_train,
                                                                          [n_train_in_quick,
                                                                           n_not_used_for_train_in_quick])

    n_not_used_for_test_in_quick = len(dataset_val) - n_test_in_quick
    dataset_val, dataset_val_not_used = torch.utils.data.random_split(dataset_val,
                                                                          [n_test_in_quick,
                                                                           n_not_used_for_test_in_quick])


train_loader = torch.utils.data.DataLoader(dataset_train, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset_val, **test_kwargs)

26422272it [00:01, 13913787.75it/s]                              


Extracting ../data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ../data\FashionMNIST\raw



29696it [00:00, 99360.42it/s]                           


Extracting ../data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ../data\FashionMNIST\raw



4422656it [00:00, 5928354.58it/s]                             


Extracting ../data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ../data\FashionMNIST\raw



6144it [00:00, 2052062.73it/s]          

Extracting ../data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\FashionMNIST\raw



Pel que fa al primer apartat de les knn, utilitzarem les imatges en raw, es a dir, el valor dels seus pixels. A continuació les obtindrem i les posarem en forma de matriu:

In [11]:
if quick_experiment:
    train_data = dataset_train.dataset.data[dataset_train.indices, :].reshape(-1, 28*28)
    test_data = dataset_val.dataset.data[dataset_val.indices, :].reshape(-1, 28*28)

    labels_train = dataset_train.dataset.targets[dataset_train.indices]
    labels_test = dataset_val.dataset.targets[dataset_val.indices]
else:
    train_data = dataset_train.data.reshape(-1, 28*28)
    test_data = dataset_val.data.reshape(-1, 28*28)

    labels_train = dataset_train.targets
    labels_test = dataset_val.targets

train_data = np.asarray(train_data, dtype=np.float) / 255.
test_data = np.asarray(test_data, dtype=np.float) / 255.    
    


<ipython-input-11-e17cf5af266e>:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_data = np.asarray(train_data, dtype=np.float) / 255.
<ipython-input-11-e17cf5af266e>:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_data = np.asarray(test_data, dtype=np.float) / 255.


### Experiment
Farem servir aquest funció per a que construeixi el model, hi carregui les dades, executi la cerca dels veïns més propers (per cada exemple del test, buscar en el train). Amb aquest veïns, s'hi aplica un classificador de més votat (així també podem veure com afecta a la seva performance). Finalment, calculem quants dels veïns hem trobat amb el model són els reals, respecte a les distàncies exactes calculades amb el bruteforce.

In [12]:
def experiment_knn(model, train_data, train_labels, test_data, test_labels, k=10, test_knn_truth=None):
    print("Train: {}".format(train_data.shape))
    print("Test: {}".format(test_data.shape))
    print("knn available?: {}".format(test_knn_truth is not None))
    print("Model: {}".format(model))
    build_time = model.build(train_data, train_labels)
    dist, idx, query_time = model.query(test_data, k=k)
    predicted_label = model.classify_from_idx(idx)
    accuracy = 100*torch.sum(predicted_label==test_labels)/float(len(test_labels))
    print("Accuracy: {:.2f}%".format(accuracy))

    if test_knn_truth is None:
        recall_at_k = 100
    else:
        recall_at_k = [len(np.intersect1d(idx[i,:], test_knn_truth[i,:], assume_unique=True)) 
                        for i in range(idx.shape[0])]
        recall_at_k = 100*np.mean(recall_at_k) / idx.shape[1]

    print("Recall@10: {:.2f}%".format(recall_at_k))

    return {"class": str(type(model)), 
            "parameters": model.__str__(),
            "test_samples": test_data.shape[0], 
            "dimensions": test_data.shape[1], 
            "classification_accuracy": float(accuracy),
            "k": 10,
            "recall@k": recall_at_k, 
            "build_time": build_time, 
            "query_time": query_time,
            "queries/s": 1.0 / (query_time / float(test_data.shape[0]))}, idx

## Apartat A. Búsqueda Brute Force **(2pts)**

En aquest primer apartat, el que hem de fer es calcular la distància "real" (o exacte) entre el dataset de test i el dataset de train. Per fer això, o bé ho feu amb codi, calculant les distàncies entre tots els samples, o bé fem servir la funcio una de les següents funcions de `scipy`:

* [scipy.spatial.distance.cdist](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html)
* [scipy.spatial.distance_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance_matrix.html)

Un cop calculada la matriu resultant de les distàncies, troba els veïns més propers per cada sample de test, és a dir els topK més propers. (K=10).

Per tal de fer-ho reutilitzable fàcilment, podem instanciar-ho dins la classe `BruteForce()`.

**Guardeu** les distàncies exactes i els veïns més propers per cada exemple de test. Ho farem servir per calcular el recall obtingut de les búsquedes aproximades (és a dir, quants dels veïns més propers s'han trobat)

In [13]:
all_info = []

### BruteForce Class

Implementació de la búsqueda basada en bruteforce, on es calculen les distancies entre tots els exemples.

In [14]:
from scipy.spatial import distance

class BruteForce():
    def __init__(self, params={}):
        self.params=params
        
    def __str__(self):
        return "BruteForce()"
        
    def build(self, vectors, labels):
        t1 = time.time()
        self.dimension = vectors.shape[1]
        self.vectors = vectors
        self.labels = labels
        self.unique_labels = np.unique(self.labels)

        elapsed = time.time()-t1
        print("Build. Nothing to do. Elapsed: {:.2f}s".format(elapsed))
        return elapsed
    
    def query(self, vectors, k=10):
        t1 = time.time()
        ############################
        ##         TO DO          ##
        ############################
        dist, ind = None, None

        elapsed = time.time()-t1
        print("Query Done. Elapsed: {:.2f}s".format(elapsed))
        
        return (dist, ind, elapsed)

    def classify(self, vectors, k=10):
        dist, idx, _ = self.query(vectors, k)
        return torch.as_tensor([np.argmax(np.bincount(self.labels[idx[i, :]], minlength=len(self.unique_labels))) 
                                for i in range(vectors.shape[0])])

    def classify_from_idx(self, idx):
        return torch.as_tensor([np.argmax(np.bincount(self.labels[idx[i, :]], minlength=len(self.unique_labels))) 
                                for i in range(idx.shape[0])])



Utilitzarem la funció `experiment_knn()` per a que ens faci totes les operacions amb el algoritme desitjat 

In [15]:
info, idx_brute_force = experiment_knn(BruteForce(), train_data, labels_train, test_data, labels_test)
all_info.append(info)

Train: (60000, 784)
Test: (1000, 784)
knn available?: False
Model: BruteForce()
Build. Nothing to do. Elapsed: 0.97s
Query Done. Elapsed: 0.00s


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
info

###  **PREGUNTA: Implementa el mètode `query` de la classe BruteForce.**
**Ha de retornar 3 variables. La variable distancies ordenada dels k veïns més propers, els index dels items del training més propers i el temps transcorregut. Les dues matrius tindràn la mida (n_items_test, k).**

**Com pots veure, el mètode build, s'encarrega de agafar les dades d'entrenament i posar-ho en una estructura, un index o la classe que permet llavors fer-hi búsquedes. Retorna el temps transcorregut. En aquest cas, no és necessari fer gaire res, però pels següents mètodes veureu que si.**

###  **PREGUNTA: Executa la búsqueda amb diferents `k` i comprova l'accuracy.**

## Apartat B. Búsqueda Aproximada **(4pts)**
En aquest apartat s'utilitzaran varies llibreria de búsqueda aproximada. Inicialment es faràn servir les de sklearn ([KdTree](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html) i [BallTree](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree.html)), i també provarem la de [Annoy](https://github.com/spotify/annoy). Per instal·lar aquesta última es pot fer simplement amb un `pip install annoy`.

In [ ]:
from sklearn.neighbors import KDTree

class ANN_sklearn_kdtree():
    def __init__(self, extra_param = {}):
        self.model_name = model_name
        self.vectors = None
        self.labels = None
        self.unique_labels = None
        
        self.extra_param = extra_param
        self.leaf_size = self.extra_param["leaf_size"] if "leaf_size" in self.extra_param else 40
        self.breadth_first = self.extra_param["breadth_first"] if "breadth_first" in self.extra_param else False
        self.dualtree = self.extra_param["dualtree"] if "dualtree" in self.extra_param else True

        
    def __str__(self):
        return "ANN_sklearn kdtree ({})".format(self.extra_param)
    
    def build(self, vectors, labels):
        t1 = time.time()

        ############################
        ##         TO DO          ##
        ############################
        self.model = None

        elapsed = time.time()-t1
        print("Index Built. Elapsed: {:.2f}s".format(elapsed))
        return elapsed

        
    def query(self, vectors, k=10):
        t1 = time.time()
        ############################
        ##         TO DO          ##
        ############################
        dist, ind = None, None
        elapsed = time.time()-t1
        print("Query Done. Elapsed: {:.2f}s".format(elapsed))
        return dist, ind, elapsed


    def classify(self, vectors, k=10):
        dist, idx, _ = self.query(vectors, k)
        return torch.as_tensor([np.argmax(np.bincount(labels_train[idx[i, :]], minlength=10)) 
                                for i in range(vectors.shape[0])])

    def classify_from_idx(self, idx):
        return torch.as_tensor([np.argmax(np.bincount(self.labels[idx[i, :]], minlength=10)) 
                                for i in range(idx.shape[0])])

In [ ]:
all_info.append(experiment_knn(ANN_sklearn_kdtree(extra_param={"leaf_size": 5}), 
                               train_data, labels_train, 
                               test_data, labels_test,
                               test_knn_truth=idx_brute_force)[0])
all_info.append(info)

In [ ]:
from sklearn.neighbors import BallTree

class ANN_sklearn_balltree():
    def __init__(self, extra_param = {}):
        self.vectors = None
        self.labels = None
        self.unique_labels = None
        
        self.extra_param = extra_param
        self.leaf_size = self.extra_param["leaf_size"] if "leaf_size" in self.extra_param else 40
        self.breadth_first = self.extra_param["breadth_first"] if "breadth_first" in self.extra_param else False
        self.dualtree = self.extra_param["dualtree"] if "dualtree" in self.extra_param else True

        
    def __str__(self):
        return "ANN_sklearn_balltree ({})".format(self.extra_param)
    
    def build(self, vectors, labels):
        t1 = time.time()

        ############################
        ##         TO DO          ##
        ############################
        self.model = None
        
        elapsed = time.time()-t1
        print("Index Built. Elapsed: {:.2f}s".format(elapsed))
        return elapsed

        
    def query(self, vectors, k=10):
        t1 = time.time()
        ############################
        ##         TO DO          ##
        ############################
        dist, ind = None, None
        elapsed = time.time()-t1
        print("Query Done. Elapsed: {:.2f}s".format(elapsed))
        return dist, ind, elapsed


    def classify(self, vectors, k=10):
        dist, idx, _ = self.query(vectors, k)
        return torch.as_tensor([np.argmax(np.bincount(labels_train[idx[i, :]], minlength=10)) 
                                for i in range(vectors.shape[0])])

    def classify_from_idx(self, idx):
        return torch.as_tensor([np.argmax(np.bincount(self.labels[idx[i, :]], minlength=10)) 
                                for i in range(idx.shape[0])])


In [ ]:
all_info.append(experiment_knn(ANN_sklearn('balltree', extra_param={"leaf_size": 5}), 
                               train_data, labels_train, 
                               test_data, labels_test,
                               test_knn_truth=idx_brute_force)[0])

In [ ]:
from annoy import AnnoyIndex
import tqdm
import sys

class ANN_annoy():
    def __init__(self, extra_param = {}):
        self.model = None
        self.extra_param = extra_param
        self.n_trees = self.extra_param["n_trees"] if "n_trees" in self.extra_param else 10
        self.k_search = self.extra_param["k_search"] if "k_search" in self.extra_param else -1

        
    def __str__(self):
        return "ANN Annoy ({})".format(self.extra_param)
    
    def build(self, vectors, labels):
        t1 = time.time()
        ############################
        ##         TO DO          ##
        ############################
        
        self.model = None
        
        elapsed = time.time()-t1
        print("Index Built. Elapsed: {:.2f}s".format(elapsed))
        return elapsed


        
    def query(self, vectors, k=10):
        t1 = time.time()
        ############################
        ##         TO DO          ##
        ############################

        dist, ind = None, None
        
        elapsed = time.time()-t1
        print("Query Done. Elapsed: {:.2f}s".format(elapsed))
        return dist, ind, elapsed


    def classify(self, vectors, k=10):
        dists, idxs, _ = self.query(vectors, k)
        return torch.as_tensor([np.argmax(np.bincount(labels_train[idxs[i]], minlength=10)) 
                                for i in range(vectors.shape[0])])

    def classify_from_idx(self, idx):
        return torch.as_tensor([np.argmax(np.bincount(self.labels[idx[i, :]], minlength=10)) 
                                for i in range(idx.shape[0])])
    

In [ ]:
all_info.append(experiment_knn(ANN_annoy(extra_param={"n_trees":10, "k_search": -1}), 
                         train_data, labels_train, test_data, labels_test,
                         test_knn_truth=idx_brute_force)[0])

###  **PREGUNTA: Implementa els mètode `build` i `query` de la classe `ANN_sklearn_kdtree`. Compara el resultat amb bruteforce.**


###  **PREGUNTA: Implementa els mètode `build` i `query` de la classe `ANN_sklearn_balltree`. Compara el resultat amb bruteforce i els altres ANN.**


###  **PREGUNTA: Implementa els mètode `build` i `query` de la classe `ANN_annoy`. Compara el resultat amb bruteforce.**


## Apartat C. Net Encoding **(2pts)**
En aquest apartat, el que farem servir és convertir les dades d'entrada (en el cas anterior el valors dels pixels de les imatges convertides en un array unidimensionals) a unes representacions corresponents a extreure els valors intermitjos al haver-hi aplicat una xarxa neuronal. 

Farem servir les que vàrem aprendre sobre el mateix dataset la sessió anterior, però la gràcia és que el mètode es podria aplicar (segurament amb pitjors resultats) per una xarxa genèrica apresa amb altres dades (pero del mateix àmbit). Com veureu, les característiques del apartat anterior tenen una dimensionalitat de 784, és a dir els 28*28 pixels. 

Són dades prou fàcils, sense soroll de fons, de mida 'tractable'.. Si ho apliquèssim a imatges reals de color.. la mida seria molt més gran (per exemple, de 300\*300\*3 obtindriem uns exemples de dimensionalitat 270.000, o de 640\*480\*3 tindriem 921.600). El fet de codificar les imatges amb característiques extretes de xarxes neuronal permet reduir moltissim la dimensionalitat, però a més, ho aconseguim capturant informació d'alt nivell en la representació.

Si no teniu un model après del exercici anterior, podem executar el següent model i n'obtindrem un que ja ens servirà

In [ ]:
model = FashionCNN()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_train, loss_test, acc_test, elapsed = experiment(model, 
                                                      device, 
                                                      loss, 
                                                      optimizer, 
                                                      train_loader, 
                                                      test_loader, 
                                                      name='#', 
                                                      save_model=False)


Hi ha varies formes d'extreure informació de les capes intermitjes d'una capa:
* aplicar [forward_hooks](https://discuss.pytorch.org/t/how-to-register-forward-hooks-for-each-module/43347/5)
* en el forward, [retornar varies variables](https://discuss.pytorch.org/t/accessing-intermediate-layers-of-a-pretrained-network-forward/12113/4) (no només la sortida final)
* en el forward, retornar la capa que ens interessa.

Aquí farem servir la última opció, per fer-ho més fàcil. Amb el model après, carregarem els pesos en una nova definicio de xarxa que no executi tots els passos. Ho farem amb la següent funció: (aquelles capes que es diuen igual, s'hi copien els pesos)

In [ ]:
def convert_model_to_extract_features(input_model):
    output_model = FashionCNN_feature_extraction()
    output_model.load_state_dict(model.state_dict())
    return output_model

In [ ]:
new_model = convert_model_to_extract_features(model)

In [ ]:
def extract_features(model, device, data_loader):
    model.eval()
    all_features = []
    
    with torch.no_grad():
        t = tqdm.tqdm(data_loader, total=len(data_loader))
        t.set_description('Extract Features ')
        for data, target in t:
            data, target = data.to(device), target.to(device)
            output = model(data)
            all_features.append(np.asarray(output))

        all_features = np.vstack(all_features)
    return all_features

In [ ]:
train_data_features = extract_features(new_model, device, train_loader)
test_data_features = extract_features(new_model, device, test_loader)

In [ ]:
print(train_data_features.shape)
print(test_data_features.shape)

In [ ]:
info, idx_brute_force_features = experiment_knn(BruteForce(), 
                                                train_data_features, labels_train, 
                                                test_data_features, labels_test)
all_info.append(info)

###  **PREGUNTA: Aprèn un model i extreu les caracteristiques de la penúltima capa.**


###  **PREGUNTA: Modifica la dimensionalitat del model de la capa fc2 (un amb més dimensions i un altre amb menys) i reaprèn els models i compara'ls.**


## Apartat D. Net Encoding. Búsqueda BruteForce i Aproximada **(3pts)**

Finalment, referem les búsquedes dels apartats A i B, però utilitzant les dades aconseguides en el apartat C. D'aquesta forma, veurem com ha influit la reducció de dimensionalitat en els mètodes.

Un cop tinguem tots els resultats, el que farem és mostrar-los gràficament, on a les X hi trobem el recall aconseguit i a les Y el temps que ha trigat en fer les N queries.

###  **PREGUNTA: Compara els resultats del bruteforce i els 3 ANN utilitzant les caracteristiques de la xarxa. Explica les diferències respecte utilitzant les dades originals.**

###  **PREGUNTA: Executa cerques amb varis paràmetres dels ANN.**


###  **PREGUNTA: Mostra una gràfica mostrant les queries/s respecte el recall que aconsegueixen.**
